In [49]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output, State
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from tensorflow.keras.models import load_model
from sklearn.preprocessing import MinMaxScaler

from tensorflow.keras.models import model_from_json

import mlutils
import pickle

In [40]:
import pandas as pd
merged_df = pd.read_csv('data/merged_df.csv')

import pymongo
from pymongo import MongoClient

# # Connect to MongoDB
client = MongoClient('mongodb://localhost:27017/')  # MongoDB connection string

In [41]:
def get_all_ticker_data(ticker):
    print("Pulling raw ticker data")
    merged_df[merged_df['ticker'] == ticker]
    return merged_df[merged_df['ticker'] == ticker]

    db = client['stocks']
    collection = db['prepared_data']
    data = collection.find({"ticker": ticker})
    return pd.DataFrame(data)

In [47]:
def predict_RNN(ticker_df, ticker, seq_length, pred_dates_range):
    # Load the pre-trained model
    model = load_model(f'models/rnn/{ticker}_{seq_length}.keras')
    if model:
        print(f"RNN Model loaded for {ticker}_{seq_length}.keras")

    data, labels = mlutils.prep_ticker_data(ticker, ticker_df)
    _, X_test, _, _, scaler, split = mlutils.scale_and_split(labels, seq_length)

    predictions = model.predict(X_test)
    predictions = scaler.inverse_transform(predictions)

    # Create the Plotly figure
    fig = go.Figure()

    # Add the actual closing price trace
    fig.add_trace(go.Scatter(
        x=ticker_df['date'],
        y=ticker_df['close'][seq_length:],
        mode='lines',
        name='Actual Closing Price',
        line=dict(color='blue')
    ))

    # Add the predicted closing price trace
    fig.add_trace(go.Scatter(
        x=ticker_df['date'][split + seq_length:],
        y=predictions.flatten(),
        mode='lines',
        name='Predicted Closing Price',
        line=dict(color='green')
    ))

    # Add the predicted closing price trace for new dates
    new_dates = pd.date_range(ticker_df['date'][split + seq_length:].iloc[-1], periods=pred_dates_range, freq='D')
    predictions_new = model.predict(X_test)
    predictions_new = scaler.inverse_transform(predictions_new)
    fig.add_trace(go.Scatter(
        x=new_dates,
        y=predictions.flatten(),
        mode='lines',
        name='Predicted New Dates',
        line=dict(color='red')
    ))

    # Update the layout
    fig.update_layout(
        title=f'{ticker} Closing Price Prediction',
        xaxis_title='Date',
        yaxis_title='Closing Price',
        legend=dict(x=0, y=1)
    )

    return f"Predicted closing price for {ticker}: {predictions_new[-1][0]:.2f}", fig

In [44]:
steps = 14

def predict_Exo(t_df, ticker, seq_length, pred_dates_range):
    t_df = t_df.sort_values('date')
    t_df['date'] = pd.to_datetime(t_df['date'])
    data = t_df.set_index('date')
    data = data.asfreq('D')
    data = data.sort_index()
    data = data.fillna(method='ffill')
    data_train = data[:-steps]
    data_test  = data[-steps:]
    
    # Load the model from a file
    with open('forecaster_autoreg.pkl', 'rb') as file:
        loaded_forecaster = pickle.load(file)
        print("Model loaded successfully!")

    predictions = loaded_forecaster.predict(steps=steps, exog=data_test[['bert_score', 'bert_label', 'num_tweets']])


In [48]:
# Initialize the Dash app
app = dash.Dash(__name__)

dropdown_options = ['TSLA', 'AAPL', 'GOOG', 'AMZN', 'MSFT']

app.layout = html.Div([
    html.H1("Stock Price Prediction Dashboard"),
    
    html.Div([
        html.Div([
        html.Label("Ticket Symbol:"),
        dcc.Dropdown(
            id='stock-dropdown',
            options=dropdown_options,
            value='AAPL',  # Default value
            placeholder="Select a stock"
        )], style={'width': '33%'}),
        html.Div([
            html.Label("Sequence Length"),
            dcc.Dropdown(id='seq-length-input', options=[1, 3, 7], value=7),
        ], style={'width': '33%'}),
        html.Div([
            html.Label("Prediction Days Ahead"),
            dcc.Input(id='predict-dates-input', type='number', value=40),
        ], style={'width': '33%'}),
    ], style={'margin-top': '20px', 'display': 'flex', 'width': '100%'}),

    html.Button('Predict', id='predict-button', n_clicks=0),

    # Output Graphs
    html.Div([
        html.Div(id='prediction-output'),
        dcc.Graph(id='prediction-graph-rnn')
    ], style={'margin-top': '20px', 'width': '100%'})
], style={'background': 'white', 'margin': 'auto'})

@app.callback(
    [Output('prediction-output', 'children'),
     Output('prediction-graph-rnn', 'figure')],

    [Input('predict-button', 'n_clicks')],
    [Input('stock-dropdown', 'value'),
     Input('seq-length-input', 'value'),
     Input('predict-dates-input', 'value')]
)
def update_prediction(n_clicks, ticker, seq_length, pred_dates_range):
    if n_clicks == 0:
        return "", {}

   # Load the raw data for graphing
    ticker_df = get_all_ticker_data(ticker)

    pred_price, graph_rnn = predict_RNN(ticker_df, ticker, seq_length, pred_dates_range)
    return pred_price, graph_rnn

if __name__ == '__main__':
    app.run_server(debug=True)

Pulling raw ticker data
RNN Model loaded for AAPL_7.keras
2/2 [==============================] - 0s 109ms/step
